* Linear Regression is a statistical technique used to find the relationship between variables
* In ML context, linear regression finds the relationship between features and a label
* In algebraic terms, the model is defined as y = mx + b
    * y is the value we want to predict
    * m is the slope of the line
    * x is the input value
    * b is the y-intercept
* In ML terms, the model is defined as y' = b + w1*x1
    * y' is the predicted value
    * b is the bias term
    * w1 is the weight for feature x1
    * x1 is the input value

<img src="../pics/linear-equation.png" alt="Mathematical representation of a linear model" width="300"/>